# boundary preserverance
1. knn preserving evaluation

## load data

In [1]:
import torch
import math
import numpy as np

In [2]:
# hyperparameters
data_shape = (2048,)

n_hidden = 500
dim_img = 2048  # number of pixels for a gap layer
dim_z = 2

# train
n_epochs = 500
batch_size = 200
learn_rate = 0.001

### train_data

In [3]:
train_data = np.load("train_data.npy")
train_pred_labels = np.load("train_pred_labels.npy")
train_data.shape, train_pred_labels.shape

((50000, 2048), (50000,))

### test_data

In [4]:
test_data = np.load("test_data.npy")
test_pred_labels = np.load("test_pred_labels.npy")
test_data.shape, test_pred_labels.shape

((10000, 2048), (10000,))

### augmentation_data

In [5]:
border_center = np.load("border_center.npy")
border_center_labels= np.load("border_center_labels.npy")
border_center.shape, border_center_labels.shape

((5000, 2048), (5000,))

## border preserving before DBP
1. M_<CP,CP>(original)
2. M_<CP,CP>(core)
3. M_<CP,CP>(core) rho=0
4. M_<CP,DP>(core)
5. M_<CP,CP><CP,DP>(core)
6. M_<bw_Complex>(core) k=15
7. M_<bw_Complex>(core) k=30

In [6]:
# define encoder
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# test encoder/decoder		

### M_<CP,CP>(original)

In [7]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_tptp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_tptp


#### train

In [ ]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [11]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.42742, 8.0, 0.0)

In [12]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(3.94338, 17.0, 0.0)

In [13]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(7.5892, 22.0, 0.0)

In [14]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(14.10516, 35.0, 0.0)

#### test

In [8]:
# test
low_center = load_encoder(border_center).cpu().numpy()
low_test = load_encoder(test_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

Instructions for updating:
Use tf.identity instead.


In [9]:
_, high_ind = high_tree.query(test_data, k=10) 
_, low_ind = low_tree.query(low_test, k=10)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.4654, 8.0, 0.0)

In [10]:
_, high_ind = high_tree.query(test_data, k=20) 
_, low_ind = low_tree.query(low_test, k=20)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(3.9487, 17.0, 0.0)

In [11]:
_, high_ind = high_tree.query(test_data, k=30) 
_, low_ind = low_tree.query(low_test, k=30)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(7.4919, 23.0, 0.0)

In [12]:
_, high_ind = high_tree.query(test_data, k=50) 
_, low_ind = low_tree.query(low_test, k=50)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(13.8663, 35.0, 0.0)

### M_<CP,CP>(core)

In [7]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptp


#### train

In [28]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [18]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.44746, 9.0, 0.0)

In [19]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(4.5924, 16.0, 0.0)

In [20]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(8.17968, 22.0, 0.0)

In [21]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(15.64786, 37.0, 0.0)

#### test

In [8]:
low_center = load_encoder(border_center).cpu().numpy()
low_test = load_encoder(test_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

Instructions for updating:
Use tf.identity instead.


In [15]:
_, high_ind = high_tree.query(test_data, k=10) 
_, low_ind = low_tree.query(low_test, k=10)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.5001, 8.0, 0.0)

In [13]:
_, high_ind = high_tree.query(test_data, k=20) 
_, low_ind = low_tree.query(low_test, k=20)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(4.5695, 16.0, 0.0)

In [14]:
_, high_ind = high_tree.query(test_data, k=30) 
_, low_ind = low_tree.query(low_test, k=30)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(8.1033, 21.0, 0.0)

In [9]:
_, high_ind = high_tree.query(test_data, k=50) 
_, low_ind = low_tree.query(low_test, k=50)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(15.4593, 35.0, 0.0)

### M_<CP,CP>(core) rho=0

In [10]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptp_rho0")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptp_rho0


#### train

In [ ]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [25]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.39768, 9.0, 0.0)

In [29]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(4.6141, 16.0, 0.0)

In [30]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(8.24208, 24.0, 0.0)

In [31]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(16.2586, 40.0, 0.0)

#### test

In [11]:
low_center = load_encoder(border_center).cpu().numpy()
low_test= load_encoder(test_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [12]:
_, high_ind = high_tree.query(test_data, k=10) 
_, low_ind = low_tree.query(low_test, k=10)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.4216, 9.0, 0.0)

In [13]:
_, high_ind = high_tree.query(test_data, k=20) 
_, low_ind = low_tree.query(low_test, k=20)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(4.5361, 16.0, 0.0)

In [14]:
_, high_ind = high_tree.query(test_data, k=30) 
_, low_ind = low_tree.query(low_test, k=30)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(8.0168, 24.0, 0.0)

In [15]:
_, high_ind = high_tree.query(test_data, k=50) 
_, low_ind = low_tree.query(low_test, k=50)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp core rho=0 K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(15.7605, 39.0, 0.0)

### M_<CP,DP>(core)

In [16]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tpdp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tpdp


#### train

In [ ]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [29]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(0.51554, 10.0, 0.0)

In [19]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(1.45436, 18.0, 0.0)

In [20]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(2.43646, 28.0, 0.0)

In [21]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(5.22802, 46.0, 0.0)

#### test

In [17]:
low_center = load_encoder(border_center).cpu().numpy()
low_test = load_encoder(test_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [18]:
_, high_ind = high_tree.query(test_data, k=10) 
_, low_ind = low_tree.query(low_test, k=10)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(0.6751, 10.0, 0.0)

In [19]:
_, high_ind = high_tree.query(test_data, k=20) 
_, low_ind = low_tree.query(low_test, k=20)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(1.8544, 18.0, 0.0)

In [20]:
_, high_ind = high_tree.query(test_data, k=30) 
_, low_ind = low_tree.query(low_test, k=30)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(3.1039, 27.0, 0.0)

In [21]:
_, high_ind = high_tree.query(test_data, k=50) 
_, low_ind = low_tree.query(low_test, k=50)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tpdbp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(6.4018, 46.0, 0.0)

### M_<CP,CP><CP,DP>(core)

In [22]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptptpdp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptptpdp


#### train

In [ ]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [12]:
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.31982, 9.0, 0.0)

In [15]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(3.63902, 16.0, 0.0)

In [13]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(6.4789, 24.0, 0.0)

In [14]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(12.9191, 38.0, 0.0)

#### test

In [23]:
low_center = load_encoder(border_center).cpu().numpy()
low_test = load_encoder(test_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [24]:
_, high_ind = high_tree.query(test_data, k=10) 
_, low_ind = low_tree.query(low_test, k=10)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.3789, 9.0, 0.0)

In [25]:
_, high_ind = high_tree.query(test_data, k=20) 
_, low_ind = low_tree.query(low_test, k=20)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(3.7013, 16.0, 0.0)

In [26]:
_, high_ind = high_tree.query(test_data, k=30) 
_, low_ind = low_tree.query(low_test, k=30)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(6.5431, 23.0, 0.0)

In [27]:
_, high_ind = high_tree.query(test_data, k=50) 
_, low_ind = low_tree.query(low_test, k=50)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp tpdbp core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(12.9314, 38.0, 0.0)

### M_<new_Complex, k=15> (core)

In [28]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_15")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_15


#### train

In [ ]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [22]:
# k=15
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(2.8041, 10.0, 0.0)

In [23]:
# k=15
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(7.8586, 20.0, 0.0)

In [24]:
# k=15
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(12.90748, 28.0, 0.0)

In [25]:
# k=15
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(22.87476, 46.0, 0.0)

#### test

In [29]:
low_center = load_encoder(border_center).cpu().numpy()
low_test = load_encoder(test_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [30]:
# k=15
_, high_ind = high_tree.query(test_data, k=10) 
_, low_ind = low_tree.query(low_test, k=10)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(2.8372, 10.0, 0.0)

In [31]:
# k=15
_, high_ind = high_tree.query(test_data, k=20) 
_, low_ind = low_tree.query(low_test, k=20)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(7.8177, 20.0, 0.0)

In [32]:
# k=15
_, high_ind = high_tree.query(test_data, k=30) 
_, low_ind = low_tree.query(low_test, k=30)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(12.7345, 28.0, 0.0)

In [33]:
# k=15
_, high_ind = high_tree.query(test_data, k=50) 
_, low_ind = low_tree.query(low_test, k=50)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=50
border_pres.mean(), border_pres.max(),border_pres.min()

(22.6303, 46.0, 0.0)

### M_<new_Complex, k=30> (core)

In [34]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_30")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_30


#### train

In [ ]:
low_center = load_encoder(border_center).cpu().numpy()
low_train = load_encoder(train_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [15]:
# k=30
_, high_ind = high_tree.query(train_data, k=10) 
_, low_ind = low_tree.query(low_train, k=10)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.17478, 9.0, 0.0)

In [16]:
_, high_ind = high_tree.query(train_data, k=20) 
_, low_ind = low_tree.query(low_train, k=20)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(6.33074, 19.0, 0.0)

In [17]:
_, high_ind = high_tree.query(train_data, k=30) 
_, low_ind = low_tree.query(low_train, k=30)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(14.1065, 29.0, 0.0)

In [18]:
_, high_ind = high_tree.query(train_data, k=50) 
_, low_ind = low_tree.query(low_train, k=50)
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(26.6353, 46.0, 0.0)

#### test

In [35]:
low_center = load_encoder(border_center).cpu().numpy()
low_test = load_encoder(test_data).cpu().numpy()
from sklearn.neighbors import KDTree
high_tree = KDTree(border_center) 
low_tree = KDTree(low_center)

In [36]:
# k=30
_, high_ind = high_tree.query(test_data, k=10) 
_, low_ind = low_tree.query(low_test, k=10)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(1.3536, 9.0, 0.0)

In [37]:
_, high_ind = high_tree.query(test_data, k=20) 
_, low_ind = low_tree.query(low_test, k=20)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=20
border_pres.mean(), border_pres.max(),border_pres.min()

(6.4749, 19.0, 0.0)

In [38]:
_, high_ind = high_tree.query(test_data, k=30) 
_, low_ind = low_tree.query(low_test, k=30)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=30
border_pres.mean(), border_pres.max(),border_pres.min()

(13.903, 29.0, 0.0)

In [39]:
_, high_ind = high_tree.query(test_data, k=50) 
_, low_ind = low_tree.query(low_test, k=50)
border_pres = np.zeros(len(test_data))
for i in range(len(test_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# new complex core K=10
border_pres.mean(), border_pres.max(),border_pres.min()

(26.1882, 46.0, 0.0)